### Ingest races.csv file

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

##### Step 1 - Read the csv using spark Dataframe Reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
races_schema = StructType(fields = [StructField("raceId", IntegerType(), False),
                                   StructField("year", IntegerType(), True),
                                   StructField("round", IntegerType(), True),
                                   StructField("circuitId", IntegerType(), True),
                                   StructField("name", StringType(), True),
                                   StructField("date", DateType(), True),
                                   StructField("time", StringType(), False),
                                   StructField("url", StringType(), False),
                                ])

In [0]:
races_df = spark.read \
    .option("header", True) \
    .schema(races_schema) \
    .csv(f"{raw_folder_path}/{v_file_date}/races.csv")

In [0]:
# races_df.printSchema()

root
 |-- raceId: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- circuitId: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- url: string (nullable = true)



In [0]:
# display(races_df.limit(5))

raceId,year,round,circuitId,name,date,time,url
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix


##### Step 2 - Add ingestion_date and race_timestamp to the dataframe

In [0]:
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [0]:
races_with_timestamp_df = add_ingestion_date(races_df) \
    .withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

##### Step 3 - Select only the required columns and rename columns

In [0]:
races_selected_df = races_with_timestamp_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), col('round'), 
                                                   col('circuitId').alias('circuit_id'), col('name'), col('race_timestamp'), 
                                                   col('ingestion_date'), col('data_source'), col('file_date'))

##### Step 4 - Write output data to processed container

In [0]:
races_selected_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.races")

In [0]:
# races_selected_df.write.mode('overwrite').partitionBy('race_year').parquet(f'{processed_folder_path}/races')

In [0]:
%sql
SELECT * FROM f1_processed.races
LIMIT 5;

race_id,race_year,round,circuit_id,name,race_timestamp,ingestion_date,data_source,file_date
1,2009,1,1,Australian Grand Prix,2009-03-29T06:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
2,2009,2,2,Malaysian Grand Prix,2009-04-05T09:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
3,2009,3,17,Chinese Grand Prix,2009-04-19T07:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
4,2009,4,3,Bahrain Grand Prix,2009-04-26T12:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18
5,2009,5,4,Spanish Grand Prix,2009-05-10T12:00:00Z,2023-12-22T11:00:50.106Z,Ergast,2021-04-18


In [0]:
dbutils.notebook.exit("Success")